In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import os
import warnings
print(os.listdir("../input"))


warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:20,.2f}'.format
%matplotlib inline

['road-weather-information-stations.csv', 'RWIS Metadata.docx', 'socrata_metadata.json']


In [2]:
data = pd.read_csv("../input/road-weather-information-stations.csv", parse_dates=[2])

 Let's check the columns and their types

In [ ]:
data.shape

In [ ]:
data.head()

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31642625 entries, 2014-03-03 12:42:00 to 2018-04-16 09:30:00
Data columns (total 8 columns):
StationName               object
DateTime                  datetime64[ns]
RecordId                  int64
RoadSurfaceTemperature    float64
AirTemperature            float64
year                      int64
month                     int64
day                       int64
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 2.7+ GB


In [ ]:
data.describe()

In [ ]:
data.StationName.value_counts()

In [ ]:
data.StationLocation.value_counts()

**What information is changing relatively quickly (every day or hour)?**

AirTemperature and RoadTemperature were recorded every minute since 2014 in each Station, So I would like to contrast the trend against other months and years.


*What information is the most important to your mission?*
The AirTemperature and RoadTemperature in each Station

*What will affect the choices you or others will need to make?*


In [3]:
data = data.set_index("DateTime", drop=False)
data["year"] = data.DateTime.dt.year
data["month"] = data.DateTime.dt.month
data["day"] = data.DateTime.dt.day

In [ ]:
data.head()

The variable *StationLocation* should be splitted so as not to miss the opportunity to use latitude and longitude

In [ ]:
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(data[data.month==2],col="year", row="month")

#Plotting AirTemperature and RoadSurfaceTemperature splitted by Year month along days
g.map(sns.lineplot, "day","AirTemperature", legend=False, color="skyblue", markers=True, dashes=False)
g.map(sns.lineplot, "day","RoadSurfaceTemperature", legend=False, color="saddlebrown", dashes=True,  markers=True)
g.fig.subplots_adjust(hspace=-.25)

#Setting the axis
g.set(ylabel="Temperature")
g.set(xlabel="Days")
g.set(xticks="")
g.despine(bottom=False, left=True)

# Customizing the legend 
legend = plt.legend(["AirTemperature","RoadSurfaceTemperature"],loc=5,facecolor="inherit")
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('black')

plt.show()

# Choropleth - Geopandas 

In [ ]:
data['StationLocation'].apply(pd.Series)

In [4]:
dictionaries_stationlocation = data["StationLocation"].apply(lambda x: eval(x))

In [6]:
data.drop("StationLocation", axis=1, inplace=True)

In [20]:
def downcast_dtypes(df):   
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    return df

In [22]:
data = downcast_dtypes(data)

In [23]:
data["longitude"] = dictionaries_stationlocation.apply(lambda x: x["longitude"])

In [25]:
data["latitude"] = dictionaries_stationlocation.apply(lambda x: x["latitude"])

In [30]:
dictionaries_stationlocation = 0

In [26]:
data = downcast_dtypes(data)

In [28]:
data.head()

,StationName,DateTime,RecordId,RoadSurfaceTemperature,AirTemperature,year,month,day,longitude,latitude
DateTime,,,,,,,,,,
2014-03-03 12:42:00,35thAveSW_SWMyrtleSt,2014-03-03 12:42:00,672560,53.88,53.88,2014,3,3,-122.37658,47.53918
2014-03-03 12:43:00,35thAveSW_SWMyrtleSt,2014-03-03 12:43:00,672561,54.05,54.05,2014,3,3,-122.37658,47.53918
2014-03-03 12:44:00,35thAveSW_SWMyrtleSt,2014-03-03 12:44:00,672562,54.21,54.21,2014,3,3,-122.37658,47.53918
2014-03-03 12:45:00,35thAveSW_SWMyrtleSt,2014-03-03 12:45:00,672563,54.38,54.38,2014,3,3,-122.37658,47.53918
2014-03-03 12:46:00,35thAveSW_SWMyrtleSt,2014-03-03 12:46:00,672564,54.54,54.54,2014,3,3,-122.37658,47.53918


In [32]:
from shapely.geometry import Point
data['geometry'] = data.apply(lambda x: Point((x.longitude, x.latitude)),axis = 1)

OSError: Could not find lib c or load any of its variants [].